In [1]:
cd C:\Users\stfu_\OneDrive\Desktop\BigData\lab1\my-mle-template

C:\Users\stfu_\OneDrive\Desktop\BigData\lab1\my-mle-template


In [80]:
import configparser
config = configparser.ConfigParser()

config['data.raw'] = {
    'data1': 'data/winequality-red.csv',
    'data2': 'data/winequality-white.csv',
    'sep': ';',
    'train_pers': '0.8',
}
config['data.prep'] = {
    'train_x': 'data/train_X.csv',
    'train_y': 'data/train_y.csv',
    'test_x': 'data/test_X.csv',
    'test_y': 'data/test_y.csv',
}

config['train'] = {
    'model_name':'random_forest',
    'n_estimators': 100,
    'criterion': 'gini',
    'random_state': 42,
    'save_path': 'experiments/rf.joblib'
}

# config['random_forest'] = {
#     'n_estimators': 100,
#     'criterion': 'gini',
#     'random_state': 42,
#     'save_path': 'experiments/rf.joblib'
# }

with open('config.ini', 'w') as configfile:
  config.write(configfile)

In [ ]:
import configparser
import pandas as pd
from sklearn.model_selection import train_test_split


if __name__ == '__main__':
    config = configparser.ConfigParser()
    config.sections()
    config.read('config.ini')

    data = []
    sep = config['data.raw']['sep']
    for k, path in config['data.raw'].items():
        if 'data' in k:
            data.append(pd.read_csv(path, sep=sep))
    data = pd.concat(data)
    X_train, X_test, y_train, y_test = train_test_split(
        data.iloc[:, :-1], data.iloc[:, -1],
        train_size=float(config['data.raw']['train_pers']), random_state=42
    )
    X_train.to_csv(config['data.prep']['train_x'], index=False)
    y_train.to_csv(config['data.prep']['train_y'], index=False)
    X_test.to_csv(config['data.prep']['test_x'], index=False)
    y_test.to_csv(config['data.prep']['test_y'], index=False)

In [77]:
import configparser
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import dump
import pickle

if __name__ == '__main__':
    config = configparser.ConfigParser()
    config.sections()
    config.read('config.ini')

    X_train = pd.read_csv(config['data.prep']['train_x'])
    y_train = pd.read_csv(config['data.prep']['train_y'])
    X_test = pd.read_csv(config['data.prep']['test_x'])
    y_test = pd.read_csv(config['data.prep']['test_y'])

    clf = RandomForestClassifier(n_estimators=int(config['random_forest']['n_estimators']),
                                 random_state=int(config['random_forest']['random_state']),)
    print('Training...')
    clf = clf.fit(X_train, y_train.iloc[:,0])
    print('Saving model...')
    dump(clf, config['random_forest']['save_path'])

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy:\t{accuracy}')

Training...
Saving model...
Accuracy:	0.67


In [81]:
import numpy as np

In [82]:
np.random.rand(100,100)

array([[0.40309539, 0.08967387, 0.76679153, ..., 0.14461094, 0.80064573,
        0.18359434],
       [0.6231153 , 0.78164659, 0.45591946, ..., 0.34991581, 0.17143901,
        0.18904677],
       [0.32776315, 0.2396018 , 0.11118851, ..., 0.1806755 , 0.99917684,
        0.32835527],
       ...,
       [0.73262167, 0.55983569, 0.90186463, ..., 0.44274174, 0.36947729,
        0.24795643],
       [0.59491884, 0.35177863, 0.65204271, ..., 0.99115139, 0.42298415,
        0.083295  ],
       [0.74378051, 0.19539439, 0.73829874, ..., 0.42553487, 0.3929113 ,
        0.40358083]])

In [1]:
cd ..

C:\Users\stfu_\OneDrive\Desktop\BigData\lab1\my-mle-template


In [2]:
# import pytest
from src.split import split
from src.train import train
from src.predict import calc_score
import numpy as np
import os



def test_train():
    os.makedirs("tests/tmp", exist_ok=True)
    n, m = 50, 10
    X_train = np.random.rand(n, m) * 100
    y_train = np.random.randint(10, size=n)
    clf_cfg = {
        'model_name': 'random_forest',
        'n_estimators': 100,
        'criterion': 'gini',
        'random_state': 42,
        'save_path': 'tests/tmp/rf.joblib',
    }
    train(X_train, y_train, clf_cfg)
    assert os.path.exists(clf_cfg['save_path'])
    os.rmdir(clf_cfg['save_path'])

In [3]:
test_train()

Training...
Saving model...
Training finished
